Author: Shai Sussman

Date: 26 May 2023

Description: This script initates the dashboard

Use batyam_dashboard venv
visit http://127.0.0.1:8050/ in your web browser.

In [ ]:
#Installing specific packages.
!venv/bin/python -m pip install dash==2.10.0
!venv/bin/python -m pip install jupyter_dash==0.4.2
!venv/bin/python -m pip install geopandas==0.13.0
!venv/bin/python -m pip install dash-leaflet==0.1.23
!venv/bin/python -m pip install dash-extensions==0.0.65
!venv/bin/python -m pip install dash_daq==0.5.0
!/Users/shai/anaconda3/bin/python -m pip install dash-bootstrap-components==1.4.1

In [1]:
import requests
from dash import Dash, html, dcc, Input, Output
from dash_extensions.javascript import  assign
import dash_daq as daq
import dash_bootstrap_components as dbc
from plotly.subplots import make_subplots
import dash_leaflet.express as dlx
import plotly.express as px
import plotly.graph_objects as go
import dash_leaflet as dl
import pandas as pd
import numpy as np
import json
import geopandas as gpd
from jupyter_dash import JupyterDash
import datetime
import requests
from dash_extensions.javascript import Namespace
ns = Namespace("someNamespace", "someSubNamespace")
from assets.graph import graph

In [2]:
# Pandas configuration
pd.options.display.max_columns=100

# Globals and CONSTANT
contextual_width_global = 1200
contextual_height_global = 275
map_height = '600px'
lower_fig_height = 450

# Use colab
USECOLAB = False

if USECOLAB:
    PATH_TO_POUP = '/content/assets/popup.js'
else:
    PATH_TO_POUP = 'assets/popup.js'


# Loading Files

In [3]:
# Load settings file:
response = requests.get('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/settings.json')
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load the JSON data from the response content
    settings = json.loads(response.content)
else:
    print(f"Error: {response.status_code}")
    
graph.status = settings['status']
graph.status_graph_color = settings['status_graph_color']
graph.contextual_width_global = contextual_width_global
graph.contextual_height_global = contextual_height_global
income_dict = settings['income_dict']

# Map Layers
# Statistical Stats
stat_json = json.loads(gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/statistical_tract_4326.geojson').to_json())
# Buildings in the simulation
sim_bldgs_gdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/buildings_for_dashboard_4326.geojson')
sim_blgds_json = json.loads(sim_bldgs_gdf.to_json())

# Convert filed to datetime
sim_bldgs_gdf['start_date'] = pd.to_datetime(sim_bldgs_gdf['start_date'])
sim_bldgs_gdf['end_date'] = pd.to_datetime(sim_bldgs_gdf['end_date'])


# Agents track Data
agents_track_status = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/agents_track_status.csv')

agents_stat_summary_by_year = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/yearly_stats_for_dashboard.csv')

graph.total_pop = int(np.round(agents_stat_summary_by_year.loc[agents_stat_summary_by_year.index[-1], 'total_pop'], 0))
graph.matrix_rows_cols = int(np.sqrt(graph.total_pop)) + 1
graph.agents_stat_summary_by_year = agents_stat_summary_by_year

graph.year_ranges = agents_stat_summary_by_year['year']

colorDict = settings['colorDict']


attribution = '© OpenStreetMap contributors, © CARTO'
cartoUrl = 'http://basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png'

In [4]:
if USECOLAB:
    url = 'https://shai2u.github.io/demographic_estimation_dashboard_article/BatYam_maplibre/map.html'
    r = requests.get(url, allow_redirects=True)
    open('/content/map.html', 'wb').write(r.content)


# Graphs

## Change age distribution graph

In [5]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def change_age_distribution_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_age_q1','New Comers_age_q2','New Comers_age_q3','stay_age_q1','stay_age_q2','stay_age_q3']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()


  # start modification


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_age_q1'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q1 (25%)",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_age_q2'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q2 (50%)",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_age_q3'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Average age in q3 (75%)",
      mode="lines",
      line=dict(color='royalblue', width=4)
      
  ))

  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_age_q1'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q1 (25%)",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_age_q2'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q2 (50%)",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_age_q3'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Average age in q2 (75%)",
      mode="lines",
      line=dict(color='firebrick', width=4)
      
  ))

  

  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[20,100],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  
  fig.update_layout(title="Age quarter distribution for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "Average Age"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")

  fig.update_xaxes(range = [0,len(graph.year_ranges)])
  fig.update_yaxes(range = [20,110])
  #end modification

  return fig

## Income distribution graph

In [6]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def income_distribution_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_income_q1','New Comers_income_q2','New Comers_income_q3','stay_income_q1','stay_income_q2','stay_income_q3']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()




  #start modification



  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_income_q1'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Income Q1 (25%)",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_income_q2'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Income Q2 (50%)",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_income_q3'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Income Q3 (75%)",
      mode="lines",
      line=dict(color='royalblue', width=4)
      
  ))

  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_income_q1'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Icome Q1 (25%)",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_income_q2'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Income Q2 (50%)",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_income_q3'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Income Q3 (75%)",
      mode="lines",
      line=dict(color='firebrick', width=4)
      
  ))



  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[0,40000],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  

  fig.update_layout(title="Income quarter distribution for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "Inocme (New Israeli Shekels)"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")


  fig.update_xaxes(range = [0,len(graph.year_ranges)])
  fig.update_yaxes(range = [0,40000])

  #end modification

  return fig

## Income category graph

In [7]:
#fig = px.line(agents_stat_summary_by_year, x="year"})
def income_category_graph(year_q,q_date):
  df_less_columns = agents_stat_summary_by_year[['year','New Comers_income_low_ratio','New Comers_income_medium_ratio','New Comers_income_high_ratio','stay_income_low_ratio','stay_income_medium_ratio','stay_income_high_ratio']].copy()
  end_index = df_less_columns[df_less_columns['year']==year_q].index.values[0]
  selected_indexes = range(0,end_index+1)
  df_for_graph = df_less_columns[df_less_columns.index.isin(selected_indexes)]

  fig = go.Figure()


  #start modification



  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_income_low_ratio'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Low Inocme",
      mode="lines",
      line=dict(color='royalblue', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_income_medium_ratio'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="Medium Income",
      mode="lines",
      line=dict(color='royalblue', width=3)
  ))



  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['New Comers_income_high_ratio'],
      legendgroup="New Comers",  # this can be any string, not just "group"
      legendgrouptitle_text="New Comers",
      name="High Income",
      mode="lines",
      line=dict(color='royalblue', width=5)
  ))

  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_income_low_ratio'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Low Income",
      mode="lines",
      line=dict(color='firebrick', width=1)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_income_medium_ratio'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="Medium Income",
      mode="lines",
      line=dict(color='firebrick', width=3)
  ))


  fig.add_trace(go.Scatter(
      x=graph.year_ranges,
      y=df_for_graph['stay_income_high_ratio'],
      legendgroup="Staying",  # this can be any string, not just "group"
      legendgrouptitle_text="Staying",
      name="High Income",
      mode="lines",
      line=dict(color='firebrick', width=5)
  ))




  fig.add_trace(go.Scatter(x=[q_date, q_date], y=[0,1],
              name=q_date,
              legendgroup="Reference",
              legendgrouptitle_text="Reference",
              mode="lines",
              line=dict(color="LightSeaGreen", width=2,dash="dashdot")))
  

  fig.update_layout(title="Income by class for Staying vs New Comers",template='plotly_white',yaxis = {'title' : "ratio"},margin={"r":0,"t":35,"l":0,"b":35,"pad":0},
                    width=contextual_width_global,height=contextual_height_global,legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01))
  fig.update_layout(hovermode="x unified")


  fig.update_xaxes(range = [0,len(graph.year_ranges)])
  fig.update_yaxes(range = [0,1])


  return fig


## Construction

In [8]:



def construction_typo_graph_generator(construction_typo_v,construction_typo_d):

  try:
    v_a = construction_typo_v[construction_typo_v['project_ty']==1]['count'].values[0]
  except:
    v_a = 0
  try:
    v_r = construction_typo_v[construction_typo_v['project_ty']==2]['count'].values[0]
  except:
    v_r = 0
  try:
    v_rr = construction_typo_v[construction_typo_v['project_ty']==3]['count'].values[0]
  except:
    v_rr = 0

  try:
    d_a = construction_typo_d[construction_typo_d['project_ty']==1]['count'].values[0]
  except:
    d_a = 0
  try:
    d_r = construction_typo_d[construction_typo_d['project_ty']==2]['count'].values[0]
  except:
    d_r = 0
  try:
    d_rr = construction_typo_d[construction_typo_d['project_ty']==3]['count'].values[0]
  except:
    d_rr = 0
  fig = go.Figure()

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_a,
      domain = {'x': [0.06, 0.25], 'y': [0.7, 0.85]},
      number_font_color="#2052a7",
      
      delta = {'reference': d_a, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_r,
      domain = {'x': [0.3, 0.5], 'y': [0.7, 0.85]},
      number_font_color="#4c84cb",
      delta = {'reference': d_r, 'relative': True,'valueformat':'.2%'}))

  fig.add_trace(go.Indicator(
      mode = "number+delta",
      value = v_rr,
      domain = {'x': [0.6, 0.80], 'y': [0.7, 0.85]},
      number_font_color="#87b1eb",
      delta = {'reference': d_rr, 'relative': True,'valueformat':'.2%'}))


  # Add images
  fig.add_layout_image(
          dict(
              source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/Building_Typology_png.png",
              xref="paper", yref="paper",
              x=0, y=0,
              sizex=1, sizey=1,
              xanchor="left", yanchor="bottom",
              layer="below")
  )

  fig.update_layout(template="plotly_white",margin={"r":0,"t":0,"l":0,"b":0,"pad":0},width=600,height=250)
  return fig


## Suburst

In [9]:
def demographic_sunburst(year_q,agents_stay_age_income,colorDict):
  title_ = f'{year_q} : Population composition snapshot'
  # color_discrete_map = color_group_map_
  fig = px.sunburst(agents_stay_age_income, path=['Stay or leave','Age group','Income category'], title=title_)

  labels_text = fig.data[0].labels.tolist()
  colorLabels = tuple(colorDict[item] for item in labels_text)
  fig.data[0].marker.colors = colorLabels
  fig.update_traces(textinfo="label+percent entry")
  fig.update_layout(showlegend=False, margin=dict(l=0, r=25, t=50, b=0), width=550, height=450)
  fig.add_layout_image(
    dict(
        source="https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/graphics/legend.png",
          xref="paper", yref="paper",
          x=0.15, y=0,
          sizex=1, sizey=1,
          xanchor="left", yanchor="bottom",
          layer="below"))
  return fig


## Bubble Figure

In [10]:
def bubble_age_income_stay_time(StayAgeIncome_ReftedDate,NewComersAgeIncome_RefDate,StayAgeIncome_SelectedDate,NewComersAgeIncome_SelectedDate,ref_date_,date_):
  #Reference plot
  fig = make_subplots(rows=1, cols=2,subplot_titles=(f"Reference Date {ref_date_}", f"Current Date {date_}"))
  fig.add_trace(
      go.Scatter(x=NewComersAgeIncome_RefDate['Income category'], y=NewComersAgeIncome_RefDate['Age group'], mode="markers",name='New Comers',marker=dict(size=NewComersAgeIncome_RefDate['ratio'] * 250,opacity=0.5,color='blue'),showlegend=True,hovertemplate='Households %{text}',text = list(NewComersAgeIncome_RefDate['units'])),

      row=1, col=1
  )
  fig.add_trace(
      go.Scatter(x=StayAgeIncome_ReftedDate['Income category'], y=StayAgeIncome_ReftedDate['Age group'], mode="markers",name='Staying',marker=dict(size=StayAgeIncome_ReftedDate['ratio'] * 250,opacity=0.5,color='red'),showlegend=True,hovertemplate='Households %{text}',text = list(StayAgeIncome_ReftedDate['units'])),
      row=1, col=1
  )
  
  fig.add_trace(
      go.Scatter(x=NewComersAgeIncome_SelectedDate['Income category'], y=NewComersAgeIncome_SelectedDate['Age group'], mode="markers",name='New Comers',marker=dict(size=NewComersAgeIncome_SelectedDate['ratio'] * 250,opacity=0.5,color='blue'),showlegend=True,hovertemplate='Households %{text}',text = list(NewComersAgeIncome_SelectedDate['units'])),
      row=1, col=2
  )
  fig.add_trace(
      go.Scatter(x=StayAgeIncome_SelectedDate['Income category'], y=StayAgeIncome_SelectedDate['Age group'], mode="markers",name='Staying',marker=dict(size=StayAgeIncome_SelectedDate['ratio'] * 250 ,opacity=0.5,color='red'),showlegend=True,hovertemplate='Households %{text}',text = list(StayAgeIncome_SelectedDate['units'])),
      row=1, col=2
  )
  # text = list(StayAgeIncome_SelectedDate['units'])
  # hover_text = '%{text}'


  #hovertemplate="name: %{y}%{x}<br>number: %{marker.symbol}<extra></extra>")


  fig.update_layout(height=450, width=450, title_text="Side By Side Subplots", margin=dict(l=0, r=0, t=50, b=0),legend=dict(orientation="h", yanchor="top",y=0.99,xanchor="left",x=0.01),template='plotly_white')
  return fig


## Graph init

In [11]:
# Renters owners graph
renters_owners_fig = graph.renters_owners('2015 Q1', '2015 Q1', graph.contextual_width_global, graph.contextual_height_global)


q_date_for_dot_matrix = '2015 Q1'

dotMAtrixFig = graph.dot_matrix(q_date_for_dot_matrix)

# Construction
d = pd.to_datetime('2015-01-01')
d2 = d - datetime.timedelta(days=180)
bldgs1 = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)

bldgs_constr = bldgs1[bldgs1['status']=='Construction'].reset_index()

construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
bldgs2 = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d2 ) & (sim_bldgs_gdf['end_date']> d2 )].copy().reset_index(drop=True)

bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)

# Sunburst

year_makrs = [year for year in np.arange(2015,2031,0.5)]
years_with_q2_makrs = []
for year in year_makrs:
    d = pd.to_datetime(f'{int(year)}-07-01')
    if year % 1 == 0:
        d = pd.to_datetime(f'{int(year)}-01-01')
    years_with_q2_makrs.append(d)
    
d = pd.to_datetime('2015-01-01')
bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()
agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
population_sunburst_graph_init = demographic_sunburst(2019,agents_stay_age_income,colorDict)

# Bubble Figure

# get date from function
d = pd.to_datetime('2015-01-01')
bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

#age the agents
age_grown1 = d.year-years_with_q2_makrs[0].year
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

#set categories to agents ages
agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
#population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
agents_stay_age_income['units'] = 1
agents_stay_age_income_count = agents_stay_age_income.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
new_comers_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='New Comers'].reset_index(drop=True)
stay_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='stay'].reset_index(drop=True)
total = new_comers_age_income_count['units'].sum()+ stay_age_income_count['units'].sum()
new_comers_age_income_count['ratio'] = new_comers_age_income_count['units']/1100
stay_age_income_count['ratio'] = stay_age_income_count['units']/1100

d_ref = pd.to_datetime('2015-01-01')
bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d_ref ) & (sim_bldgs_gdf['end_date']> d_ref )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

agents_synced_buildings_ref_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
agents_synced_buildings_ref_date_stay_new = agents_synced_buildings_ref_date[agents_synced_buildings_ref_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

#age the agents
age_grown1 = d_ref.year-years_with_q2_makrs[0].year
agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] + age_grown1
agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d_ref.year - p['start_date'].year),axis=1)

#set categories to agents ages
agents_synced_buildings_ref_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_ref_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
agents_stay_age_income_ref =agents_synced_buildings_ref_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
#population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
agents_stay_age_income_ref['units'] = 1
agents_stay_age_income_ref = agents_stay_age_income_ref.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
new_comers_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='New Comers'].reset_index(drop=True)
stay_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='stay'].reset_index(drop=True)
total_ref = new_comers_age_income_count_ref['units'].sum()+ stay_age_income_count_ref['units'].sum()
new_comers_age_income_count_ref['ratio'] = new_comers_age_income_count_ref['units']/1100
stay_age_income_count_ref['ratio'] = stay_age_income_count_ref['units']/1100
bubble_age_income = bubble_age_income_stay_time(stay_age_income_count_ref,new_comers_age_income_count_ref,stay_age_income_count,new_comers_age_income_count,'2015 Q1','2015 Q1')


# Dash

In [13]:


line_style = dict(weight=2, opacity=1, color='blue', fillOpacity=0,dashArray="10 10")

classes = ['Building before', 'Construction', 'Building after']
colorscale = ['#808080', '#F9C70F', '#4473C5']
style = dict(weight=2, opacity=1, color='grey', fillOpacity=0.7)
colorbar = dlx.categorical_colorbar(categories=classes, colorscale=colorscale, width=300, height=30, position="bottomright")

#globals 


style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.props.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value == classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")


#dl.Map()

#geojson = dl.GeoJSON(data=data, options=dict(onEachFeature=ns("bindPopup")))

mapObj = dl.Map([dl.TileLayer(url=cartoUrl, maxZoom=20, attribution=attribution),dl.GeoJSON(data=stat_json, options={'style':line_style},),
                 dl.GeoJSON(data = sim_blgds_json,options = {'style':style_handle, 'onEachFeature':ns("bindPopup")}, id='simulatedBldgs',hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="status")),colorbar],center=[32.0272,34.7444], zoom=16, style={'width': '100%', 'height': map_height})

if USECOLAB:
    #url = 'https://shai2u.github.io/demographic_estimation_dashboard_article/assets/popup.js'
    #r = requests.get(url, allow_redirects=True)
    #open('/content/assets/popup.js', 'wb').write(r.content)
    !wget https://shai2u.github.io/demographic_estimation_dashboard_article/assets/popup.js
    !mkdir assets
    !mv popup.js assets/

# map3d = html.Iframe(id='ifame-cell', height=map_height, width="100%",
#                                                        src="https://technion-gis.maps.arcgis.com/apps/instant/3dviewer/index.html?appid=73d48240324e45eea139ea9e0c2fe6e1")

d = pd.to_datetime('2015-01-01')
bldgs1 = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)
bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)

if len(bldg_status)<3:
  if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
  if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
    bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)
    
bldg_status_count = graph.get_status(bldg_status)

#create an empty dataset with agent coutn
num_of_agents = 3481
MatrixPlot = pd.DataFrame({'AgentID':range(0,num_of_agents)})
#Number of Columns choose:
numOfColumns=59
numOfRows = 59
# set the columns to display the column number
MatrixPlot['x'] = list(range(1,numOfColumns+1))*numOfRows
#prepare the y column (row number)
y = [[i]*numOfColumns for i in range(1,numOfRows+1)]
temp = []
for i in range(0,numOfRows):
    temp +=y[i]
y = temp
MatrixPlot['y'] = y

agents_time_new_Comers_stay = agents_stat_summary_by_year[['year','New Comers','stay']].fillna(0)
agents_time_new_Comers_stay['emptyPalces'] = num_of_agents -(agents_time_new_Comers_stay['stay'] +agents_time_new_Comers_stay['New Comers'])


dashboard_page =  html.Div([
  html.Div([
            html.Div([
    html.Div([
            
              #To Do Add static legend, that won't chnaged based on years and statistical stats
                  dbc.Card(
        children=[
            dbc.CardHeader(["Map of the study area ",html.Span(id='3d_map_date')]),
            dbc.CardBody([html.Div(mapObj,id='map_2d',style={'display':'block','textAlign':'center'}),
                        html.Div(
                            []
                            ,id='map_3d')])]),
                  
              # html.Div([
              
              # ])
            ],style={'width': '34%', 'display': 'inline-block','textAlign':'center'}),
        
        html.Div([
                  dbc.Card(
        children=[
            dbc.CardHeader("Time series graph with context"),
            dbc.CardBody(html.Div([dcc.Graph(id='time_seiries_graph',figure=renters_owners_fig,style={'textAlign':'center'})]))])
                  ,

             html.Div([
                                          dbc.Card(
        children=[
            dbc.CardHeader("Current construction typologies"),
            dbc.CardBody([
                         dcc.Graph(id='typo_count', figure=construction_typo_graph,style={'textAlign':'center'}),
                          
            ])])],style={'width': '50%', 'display': 'inline-block','textAlign':'center'}),
              html.Div([
                        dbc.Card(
        children=[ dbc.CardHeader("Building status count"),
            dbc.CardBody([dcc.Graph(id='status_count',figure=bldg_status_count,style={'textAlign':'center'})]
                         )])
                        ],style={'width': '50%', 'display': 'inline-block'})
            ], style={'width': '66%', 'float': 'right', 'display': 'inline-block','textAlign':'center'}),
        ],style={'padding':'1px'}), # seperator between uppwer and lower!
        #-------------------------------- Lower Div --------------------------------------
        html.Div([
                  #---------------------- 2 Variables graph -------------------------------
                  html.Div([
                            html.Div([
                                      dbc.Card(
        children=[
            dbc.CardHeader("2 Variables bubble graph"),
            dbc.CardBody( dcc.Graph(id='bubble_graph',figure=bubble_age_income,style={'width': '100%','marginLeft':'auto','marginRight':'auto','textAlign':'center'}))])])],style={'width': '34%', 'display': 'inline-block','marginLeft':'auto','marginRight':'auto','textAlign':'center'}),
                      html.Div([
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                    #---------------------- sunburst -------------------------------

                                dbc.Card(
        children=[
            dbc.CardHeader("Population sunburst"),
            dbc.CardBody(dcc.Graph(id='population_sunburst_fig',figure= population_sunburst_graph_init,style={'width': '100%','marginLeft':'auto','marginRight':'auto','textAlign':'center'}))])],style={'width': '33%', 'display': 'inline-block','marginLeft':'auto','marginRight':'auto','textAlign':'center'}),
                      html.Div([
                                
                                #To Do Add static legend, that won't chnaged based on years and statistical stats
                      #---------------------- Dot Matrix -------------------------------

                                dbc.Card(
        children=[
            dbc.CardHeader("Population dot matrix"),
            dbc.CardBody(dcc.Graph(id='dot_matrix_fig',figure=dotMAtrixFig,style={'width': '100%','marginLeft':'auto','marginRight':'auto','textAlign':'center'}))])],style={'width': '33%', 'display': 'inline-block','marginLeft':'auto','marginRight':'auto','textAlign':'center'})
            ],style={'padding':'1px'}),
                                  #---------------------- Controllers -------------------------------

            html.Div([dbc.Card(
        children=[
            dbc.CardHeader("Controller"),
            dbc.CardBody([html.Div(
                      daq.BooleanSwitch(
                            on=False,
                            label="3D",
                            labelPosition="top",id='3d_map_switch'),style={'width':'10%', 'display': 'inline-block'}),
                    html.Div([dcc.Dropdown( id='select_context',options=[
                                                                             {'label': 'Owners Renters Count', 'value': 'Owners Renters Count'},
                                                                             {'label': 'Chnage Apartment Size', 'value': 'Chnage Apartment Size'},
                                                                             {'label': 'Change in Age Distribution', 'value': 'Change in Age Distribution'},
                                                                             {'label': 'Change in Income Distribution', 'value': 'Change in Income Distribution'},
                                                                             {'label': 'Change in Income Category', 'value': 'Change in Income Category'},



                                  ], value='Owners Renters Count')],style={'width':'25%', 'display': 'inline-block'}),
                  html.Div([dcc.RangeSlider(id='years-slider',
             min=2015,
             max=2030,
             value=[2015,2015],
             marks={str(year):str(year) for year in np.arange(2015,2030,1)},
                                                    step=0.5
                                                    )],style={'width':'65%', 'display': 'inline-block'})
                          ]),


                    
                  ])],style={'textAlign':'center'})
  ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'})
])
external_stylesheets = [dbc.themes.BOOTSTRAP, "https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/assets/style.css"]
app = JupyterDash(__name__,suppress_callback_exceptions=True,prevent_initial_callbacks=True, external_stylesheets=external_stylesheets)


app.title = "Population Dashboard"

#run_with_ngrok(app)

app.layout = html.Div([

    dashboard_page
])

@app.callback(
    Output('simulatedBldgs', 'data'),
    Output('status_count','figure'),
    Output('dot_matrix_fig','figure'),
    Output('time_seiries_graph','figure'),
    Output('typo_count','figure'),
    Output('population_sunburst_fig','figure'),
    Output('bubble_graph','figure'),
    Output('map_2d','style'),
    Output('map_3d','children'),
    Output('3d_map_date','children'),
    Input('years-slider', 'value'),
    Input('select_context','value'),
    Input('3d_map_switch','on')
    # Output('test','children'),
    #Output('dot_matrix_fig','figure'),

)
def update_output_div(input_value,input_select_context,d3_map_switch):
    #sim_bldgs_gdf
    ref_year = int(input_value[0])
    selected_year = int(input_value[1])
    date_return = f'Q3 {selected_year}'
    q_date_for_dot_matrix = f'{selected_year} Q3'
    d = pd.to_datetime(f'{selected_year}-07-01')
    map_libre_date = int(f'{selected_year}0701')
    if input_value[1] % 1 == 0:
        d = pd.to_datetime(f'{selected_year}-01-01')
        date_return = f'Q1 {selected_year}'
        q_date_for_dot_matrix = f'{selected_year} Q1'
        map_libre_date = int(f'{selected_year}0101')
    #reference date
    d_ref = pd.to_datetime(f'{ref_year}-07-01')
    ref_q_date = f'Q3 {ref_year}'
    ref_q_date_e = f'{ref_year} Q3'
    if input_value[0] % 1 == 0:
          d_ref = pd.to_datetime(f'{ref_year}-01-01')
          ref_q_date = f'Q1 {ref_year}'
          ref_q_date_e = f'{ref_year} Q1'

    date_for_js_site = q_date_for_dot_matrix.replace(' ','_')

    bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    bldgs['start_date'] = bldgs['start_date'].astype(str)
    bldgs['end_date'] = bldgs['end_date'].astype(str)

    if d3_map_switch:
      bldgsJson=None
      #display 2D/3D Map
      map_2d_display = {'display':'none'}
      #item = gis.content.get("732b15ecf0f848df9ab4d8c8f059ea56")
      #layer = item.layers[0]
      #new_def = {"DefinitionQuery" :f"start_date_int<= {arcgis_date}"}
      #layer.manager.update_definition(new_def)
      rand_id = random_int = np.random.randint(0,10) #to refresh 3d
      # map3d = html.Iframe(id=f'ifame-cell-{rand_id}', height=map_height, width="100%",
      #                                                  src="https://technion-gis.maps.arcgis.com/apps/instant/3dviewer/index.html?appid=73d48240324e45eea139ea9e0c2fe6e1")
      #map_3d_display = {'display':'block'}
      # #/content/scene_batyam.html
      # map3d = html.Iframe(id=f'ifame-cell-{rand_id}', height=map_height, width="100%",
      #                                             src=f"https://shai2u.github.io/demographic_estimation_dashboard_article/dashboard/js_maps/scene_batyam_{date_for_js_site}.html")
      map3d = html.Iframe(id=f'ifame-cell-{rand_id}', height=map_height, width="100%",
                                                  src=f"https://shai2u.github.io/demographic_estimation_dashboard_article/BatYam_maplibre/{map_libre_date}.html")

      #map_3d_display = {'display':'block'}
    else:
      bldgsJson = json.loads(bldgs.to_json())
      #display 2D/3D Map
      map_2d_display = {'display':'block'}
      map3d = ''
      #map_3d_display = {'display':'none'}

    #construction
    bldg_status = bldgs1['status'].value_counts().to_frame().reset_index()
    bldg_status.rename(columns={'index':'status','status':'count'},inplace=True)
    if len(bldg_status)<3:
      if len(bldg_status[bldg_status['status'].isin(['Building before'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building before'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Construction'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Construction'],'count':[0]})]).reset_index(drop=True)
      if len(bldg_status[bldg_status['status'].isin(['Building after'])])==0:
        bldg_status = pd.concat([bldg_status,pd.DataFrame({'status':['Building after'],'count':[0]})]).reset_index(drop=True)
    bldg_status_count = graph.get_status(bldg_status)
    
    d2 = d - datetime.timedelta(days=180)
    bldgs2 = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)

    bldgs_constr = bldgs2[bldgs2['status']=='Construction'].reset_index()

    construction_typo_v = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_v.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)

    bldgs3 = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d2 ) & (sim_bldgs_gdf['end_date']> d2 )].copy().reset_index(drop=True)
    bldgs_constr = bldgs3[bldgs3['status']=='Construction'].reset_index()

    construction_typo_d = bldgs_constr['project_ty'].value_counts().to_frame().reset_index()
    construction_typo_d.rename(columns={'index':'project_ty','project_ty':'count'},inplace=True)
    construction_typo_graph = construction_typo_graph_generator(construction_typo_v,construction_typo_d)
    #bldg_status_count = get_status_graph(bldg_status)


    # Dot Matrix

    dotMAtrixFig = graph.dot_matrix(q_date_for_dot_matrix)

    #time context
    #renters Owners
    if input_select_context == 'Owners Renters Count':
      context_fig = graph.renters_owners(q_date_for_dot_matrix , ref_q_date_e, graph.contextual_width_global, graph.contextual_height_global)
    elif input_select_context == 'Chnage Apartment Size':
      context_fig = graph.apartment(q_date_for_dot_matrix , ref_q_date_e, graph.contextual_width_global, graph.contextual_height_global)
    elif input_select_context == 'Change in Age Distribution':
      context_fig = graph.change_age_distribution(q_date_for_dot_matrix, ref_q_date_e, graph.contextual_width_global, graph.contextual_height_global)                                                                             
    elif input_select_context == 'Change in Income Distribution':
      context_fig = income_distribution_graph(q_date_for_dot_matrix,ref_q_date_e)
    elif input_select_context == 'Change in Income Category':
      context_fig = income_category_graph(q_date_for_dot_matrix,ref_q_date_e)                                                                           
    else:
      context_fig = graph.renters_owners(q_date_for_dot_matrix,ref_q_date_e)

    #sunburst

    bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    age_grown1 = d.year-years_with_q2_makrs[0].year
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

    agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    population_sunburst_graph = demographic_sunburst(date_return,agents_stay_age_income,colorDict)


    #bubble graphs Selected
    bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d ) & (sim_bldgs_gdf['end_date']> d )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

    agents_synced_buildings_to_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_to_date_stay_new = agents_synced_buildings_to_date[agents_synced_buildings_to_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    #age the agents
    age_grown1 = d.year-years_with_q2_makrs[0].year
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_to_date_stay_new.loc[agents_synced_buildings_to_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d.year - p['start_date'].year),axis=1)

    #set categories to agents ages
    agents_synced_buildings_to_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_to_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income =agents_synced_buildings_to_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    #population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
    agents_stay_age_income['units'] = 1
    agents_stay_age_income_count = agents_stay_age_income.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
    new_comers_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='New Comers'].reset_index(drop=True)
    stay_age_income_count = agents_stay_age_income_count[agents_stay_age_income_count['Stay or leave']=='stay'].reset_index(drop=True)
    #total = new_comers_age_income_count['units'].sum()+ stay_age_income_count['units'].sum()
    new_comers_age_income_count['ratio'] = new_comers_age_income_count['units']/1100
    stay_age_income_count['ratio'] = stay_age_income_count['units']/1100

    #reference selection
    bldgs  = sim_bldgs_gdf[(sim_bldgs_gdf['start_date']< d_ref ) & (sim_bldgs_gdf['end_date']> d_ref )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

    agents_synced_buildings_ref_date = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    agents_synced_buildings_ref_date_stay_new = agents_synced_buildings_ref_date[agents_synced_buildings_ref_date['status_x']!='Leave'].drop_duplicates().reset_index(drop=True)

    #age the agents
    age_grown1 = d_ref.year-years_with_q2_makrs[0].year
    agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='stay','age'] + age_grown1
    agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers','age'] = agents_synced_buildings_ref_date_stay_new.loc[agents_synced_buildings_ref_date_stay_new['status_x']=='New Comers'].apply(lambda p: p['age'] + (d_ref.year - p['start_date'].year),axis=1)

    #set categories to agents ages
    agents_synced_buildings_ref_date_stay_new['age_group'] = pd.cut(agents_synced_buildings_ref_date_stay_new['age'], [0,44,64,84,130],right=True, labels=["18-44", "45-64", "65-84", "85+"],ordered=True)
    agents_stay_age_income_ref =agents_synced_buildings_ref_date_stay_new[['status_x','age_group','income_cat']].reset_index(drop=True).rename(columns={'status_x':'Stay or leave','age_group':'Age group','income_cat':'Income category'})
    #population_sunburst_graph = demographic_sunburst(2019,agents_stay_age_income,colorDict)
    agents_stay_age_income_ref['units'] = 1
    agents_stay_age_income_ref = agents_stay_age_income_ref.groupby(['Stay or leave','Age group','Income category']).agg({'units':'count'}).reset_index()
    new_comers_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='New Comers'].reset_index(drop=True)
    stay_age_income_count_ref = agents_stay_age_income_ref[agents_stay_age_income_ref['Stay or leave']=='stay'].reset_index(drop=True)
    #total_ref = new_comers_age_income_count_ref['units'].sum()+ stay_age_income_count_ref['units'].sum()
    new_comers_age_income_count_ref['ratio'] = new_comers_age_income_count_ref['units']/1100
    stay_age_income_count_ref['ratio'] = stay_age_income_count_ref['units']/1100
    bubble_age_income = bubble_age_income_stay_time(stay_age_income_count_ref,new_comers_age_income_count_ref,stay_age_income_count,new_comers_age_income_count,ref_q_date,date_return)


    return bldgsJson,bldg_status_count,dotMAtrixFig,context_fig,construction_typo_graph,population_sunburst_graph,bubble_age_income,map_2d_display,map3d,q_date_for_dot_matrix

## To dos
## Add Tama 38 Construction Current Coutner

app.run_server(mode='External',host='0.0.0.0',debug=True,port=8060)

Dash is running on http://0.0.0.0:8060/

Dash app running on http://0.0.0.0:8060/
